In [1]:
import pandas as pd
from lxml import etree
import re
from tqdm.notebook import tqdm

In [2]:
# Create functions to free memory once function scope is left
def _readCorpus(corpus="Corpus/", files=None ):
    if files is None:
        files = ["malti03.parl.1.txt",
                 "malti03.parl.2.txt",
                 "malti03.parl.3.txt",
                 "malti03.parl.4.txt",
                 "malti03.parl.5.txt"]
    xml_data = []
    for file in tqdm(files, desc='Reading Files'):
        xml_data.append(open(corpus+file, 'r', encoding='utf8').read())  # Read file
    return xml_data

In [3]:
def _ParseAsXML():
    parser = etree.XMLParser(recover=True)
    roots = []
    xml_data = _readCorpus()
    for xml in tqdm(xml_data, desc='Parsing XML'):
        roots.append(etree.fromstring(xml, parser=parser))
    return roots

In [4]:
# Define a word class with some functionality
class Word:
    def __init__(self, kelma, tip, mamma, gherq):
        self.kelma = kelma
        self.tip = tip
        self.mamma = mamma
        self.gherq = gherq

    def __str__(self):
        return  'Kelma: ' + self.kelma + '\n' + \
                'tip: ' + self.tip + '\n' + \
                'mamma: ' + self.mamma + '\n' + \
                'gherq: ' + self.gherq

    def __iter__(self):
        for each in self.__dict__.values():
            yield each

    def __getitem__(self, index):
        if index < 4:
            return list(self.__dict__.values())[index]
        else:
            raise IndexError('Max index is 3')

In [7]:
# Define Sentence class with some functionality
class Sentence:
    def __init__(self, words=None):
        if words is None:
            words = []
        self.words = words

    def __str__(self):
        s = ''
        for word in self.words:
            s += word.kelma + " "
        return s

    def __iter__(self):
        for word in self.words:
            yield word

    def __getitem__(self, index):
        return self.words[index]

    def append(self, word: Word):
        self.words.append(word)

In [8]:
# Mention garbage collection effort and try outs for pd and numpy and why i settled with list of lists (pd matrix, list of lists no)
def CorpusAsListOfLists():
    roots = _ParseAsXML()
    sentences = []
    for root in tqdm(roots, desc='XML File'):
        for i, p in tqdm(enumerate(root), desc='Paragraph'):
            for k, s in enumerate(p):
                unfiltered_sentence = re.split(r'\n', s.text.lstrip('\n'))
                sentence = Sentence()
                for unfiltered_word in unfiltered_sentence:
                    if unfiltered_word is not "":
                        filtered_word = unfiltered_word.split('\t')
                        sentence.append(Word(   filtered_word[0],
                                                filtered_word[1],
                                                filtered_word[2],
                                                filtered_word[3]))
                if sentence is not []:
                    sentences.append(sentence)
    return sentences

corp = CorpusAsListOfLists()

Reading Files:   0%|          | 0/5 [00:00<?, ?it/s]

--- 8.114623785018921 seconds ---
--- 16.89559268951416 seconds ---
--- 0.4140005111694336 seconds ---


Parsing XML:   0%|          | 0/5 [00:00<?, ?it/s]

XML File:   0%|          | 0/5 [00:00<?, ?it/s]

Paragraph: 0it [00:00, ?it/s]

Paragraph: 0it [00:00, ?it/s]

Paragraph: 0it [00:00, ?it/s]

Paragraph: 0it [00:00, ?it/s]

Paragraph: 0it [00:00, ?it/s]

In [19]:
df = pd.DataFrame(corp)
print(df.head(5))
print(df.shape)
for i in range(5):
    print(corp[i])
print(len(corp))
del df

                                           0    \
0  Kelma: 1\ntip: X-DIG\nmamma: 1\ngherq: null   

                                           1     2     3     4     5     6    \
0  Kelma: .\ntip: X-PUN\nmamma: .\ngherq: null  None  None  None  None  None   

    7     8     9    ...   136   137   138   139   140   141   142   143  \
0  None  None  None  ...  None  None  None  None  None  None  None  None   

    144   145  
0  None  None  

[1 rows x 146 columns]
(3408, 146)
2 . 
3408


In [ ]:
def SequenceFrequency(corpus, n=2, limit=None, til=True):
    # remove sentences in corpus with length < n
    local_corpus = [s for s in corpus if len(s.words) >= n]

    if limit is None :
        limit = len(local_corpus)
    # Start by getting frequency of (n - 1) sized sequence is first sequence in a given sentence
    

def ngram(corpus, n=2, limit=None, til=True):
    # remove sentences in corpus with length < n
    local_corpus = [s for s in corpus if len(s.words) >= n]

    if limit is None :
        limit = len(local_corpus)